In [1]:
!pip install pysal

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 40.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 k

In [2]:
!pip install geopandas libpysal esda matplotlib

In [12]:
import numpy as np
import pandas as pd
from pysal.lib import weights
from pysal.explore import esda
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist


2.5
2.5
2.5


/usr/local/lib/python3.11/dist-packages/spaghetti/network.py:41: FutureWarning: The next major release of pysal/spaghetti (2.0.0) will drop support for all ``libpysal.cg`` geometries. This change is a first step in refactoring ``spaghetti`` that is expected to result in dramatically reduced runtimes for network instantiation and operations. Users currently requiring network and point pattern input as ``libpysal.cg`` geometries should prepare for this simply by converting to ``shapely`` geometries.
  warnings.warn(dep_msg, FutureWarning, stacklevel=1)


In [4]:
df = pd.read_excel('/content/sample_radiosonde.xlsx')

In [5]:
df.head()

,Height,Temp,Pres,Latitude,Longitude
0,257.000000,29.143333,982.718811,79.670000,18.960000
1,262.582861,29.124724,982.065887,79.671037,18.961257
2,268.165722,29.106114,981.413397,79.672074,18.962514
3,273.748583,29.087505,980.761341,79.673111,18.963771
4,279.331444,29.068895,980.109718,79.674148,18.965028


In [7]:
coords = df[['Latitude', 'Longitude', 'Height']].values


In [13]:
def compute_3d_distances(coords):
    return cdist(coords, coords)


In [14]:
distances = compute_3d_distances(coords)


In [15]:
# Create spatial weights using distance-based threshold
threshold = 100  # You can adjust this threshold based on your data
weights_matrix = distances < threshold  # True/False based on distance threshold


In [21]:
# Create the adjacency list manually
adjlist = {}
for i in range(len(coords)):
    adjlist[i] = []
    for j in range(len(coords)):
        if i != j and distances[i, j] < threshold:
            adjlist[i].append(j)

# Create a spatial weights object from the adjacency list
w = weights.W(adjlist)


In [22]:
scaler = StandardScaler()
df[['Temp', 'Pres']] = scaler.fit_transform(df[['Temp', 'Pres']])


In [23]:
# Calculate Moran's I for Temp and Pres
moran_temp = esda.Moran(df['Temp'], w)
moran_pres = esda.Moran(df['Pres'], w)


In [24]:
# Print the results for Moran's I
print(f"Moran's I for Temp: {moran_temp.I}")
print(f"Moran's I for Pres: {moran_pres.I}")

# Create Spatial Autocorrelation Matrix
spatial_autocorr_matrix = pd.DataFrame({
    'Temp': [moran_temp.I, moran_temp.I],
    'Pres': [moran_temp.I, moran_pres.I]
}, index=['Temp', 'Pres'])


Moran's I for Temp: 0.999893696540323
Moran's I for Pres: 0.9998404926545176


In [25]:
print("Spatial Autocorrelation Matrix (Moran's I values):")
print(spatial_autocorr_matrix)


Spatial Autocorrelation Matrix (Moran's I values):
          Temp      Pres
Temp  0.999894  0.999894
Pres  0.999894  0.999840
